In [51]:
import pandas as pd
from collections import defaultdict
class Transmission:
    def __init__(self,train):
        self.transmission=None  #pandas table containing all state u-> v probabilities and counts
        self.count=None #dictionary containing all u counts
        self.readTrain(train)
    def readTrain(self,train):
        file=open(train,'r',encoding='utf8')
        yseq=['START']
        for line in file:
            xy=line.split(' ')
            if len(xy)<=1:
                yseq.append('END')
                yseq.append('START')
                continue
            x,y=xy
            x=x.strip()
            y=y.strip()
            yseq.append(y)
        file.close()
        del yseq[-1]#delete the last 'start' tag from the sequence
        transDict = defaultdict(int)
        countDict= defaultdict(int)
        for i in range(len(yseq)-1):
            u=yseq[i]
            countDict[u]+=1
            if u=='END':
                continue
            v=yseq[i+1]
            if(u=='START' and v=='END'):#blank line detect
                countDict['START']-=1#remove extra count
                break
            transDict[(u,v)]+=1
        self.transmission=pd.DataFrame([(i[0], i[1], j) for i, j in transDict.items()],columns=['u','v','Count'])
        self.count=pd.DataFrame([(i, j) for i, j in countDict.items()],columns=['u','GCount'])
        self.genprob()
    def genprob(self):
        self.transmission=pd.merge(self.transmission, self.count, on='u')
        #print(self.transmission.head(2))
        #print(self.count.head(2))
        self.transmission['transmission']=self.transmission['Count']/self.transmission['GCount']
        self.transmission.drop(['GCount'], axis=1)
        
    def transmission_proba(self,u,v):
        return self.transmission.loc[(self.transmission['u'] == u) & (self.transmission['v'] == v)]['transmission'][0]

In [52]:
EN_transmission = Transmission('./EN/train')
print(EN_transmission.transmission_proba('START','B-NP'))
#print(EN_transmission.transmission_proba('B-NP','END'))
#print(EN_transmission.transmission_proba('START','START'))

0.6480490669450607
